In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))

nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [11]:
resNet50_base_model = ResNet50(include_top=False, weights='imagenet',pooling='avg')

x = resNet50_base_model.output
x = Dropout(0.5)(x)
x = Dense(10,activation='softmax')(x)

resNet50_model = Model(inputs=resNet50_base_model.input,outputs=x)

resNet50_model.summary()

for layer in resNet50_model.layers[:173]:
    layer.trainable = False
    
for i,layer in enumerate(resNet50_model.layers):
    print(i,layer.trainable,layer.name)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_3[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_99 (Activation)      (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [12]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.15,
                                         rotation_range=15,
                                         height_shift_range=0.15,
                                         width_shift_range=0.15,
                                         channel_shift_range=10,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [13]:
resNet50_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(),
                    #optimizer='rmsprop',
                    metrics=['accuracy'])

# fine-tune the model
resNet50_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/resnet50_fc'),
                                   CSVLogger('resnet50_fc'),
                                   EarlyStopping(patience=5)])

Epoch 1/10
156/156 [==============================] - 355s 2s/step - loss: 1.7254 - acc: 0.4052 - val_loss: 2.6688 - val_acc: 0.3793
Epoch 2/10
156/156 [==============================] - 345s 2s/step - loss: 1.0894 - acc: 0.6389 - val_loss: 2.8882 - val_acc: 0.3789
Epoch 3/10
156/156 [==============================] - 347s 2s/step - loss: 0.9289 - acc: 0.6922 - val_loss: 2.1490 - val_acc: 0.4708
Epoch 4/10
156/156 [==============================] - 344s 2s/step - loss: 0.8508 - acc: 0.7160 - val_loss: 2.3002 - val_acc: 0.4591
Epoch 5/10
156/156 [==============================] - 345s 2s/step - loss: 0.8008 - acc: 0.7295 - val_loss: 2.3589 - val_acc: 0.4438
Epoch 6/10
156/156 [==============================] - 347s 2s/step - loss: 0.7881 - acc: 0.7338 - val_loss: 2.4118 - val_acc: 0.4800
Epoch 7/10
156/156 [==============================] - 351s 2s/step - loss: 0.7576 - acc: 0.7450 - val_loss: 2.6085 - val_acc: 0.4303
Epoch 8/10
156/156 [==============================] - 340s 2s/step - 

In [14]:
for layer in resNet50_model.layers[:131]:
    layer.trainable = False
    
for layer in resNet50_model.layers[131:]:
    layer.trainable = True

In [15]:
resNet50_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-3, momentum=0.9,decay=1e-6,nesterov=False),
                      #optimizer=optimizers.Adam(),
#                     optimizer='rmsprop',
                    metrics=['accuracy'])

# fine-tune the model
resNet50_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/resnet50_fine_tune'),
                                   CSVLogger('resnet50_fine_tune'),
                                   EarlyStopping(patience=5)])

Epoch 1/20
156/156 [==============================] - 364s 2s/step - loss: 0.4310 - acc: 0.8573 - val_loss: 1.1075 - val_acc: 0.6471
Epoch 2/20
156/156 [==============================] - 363s 2s/step - loss: 0.1958 - acc: 0.9408 - val_loss: 1.0342 - val_acc: 0.6651
Epoch 3/20
156/156 [==============================] - 350s 2s/step - loss: 0.1289 - acc: 0.9601 - val_loss: 1.1774 - val_acc: 0.6311
Epoch 4/20
156/156 [==============================] - 346s 2s/step - loss: 0.0923 - acc: 0.9722 - val_loss: 1.2236 - val_acc: 0.6341
Epoch 5/20
156/156 [==============================] - 352s 2s/step - loss: 0.0759 - acc: 0.9780 - val_loss: 1.1385 - val_acc: 0.6555
Epoch 6/20
156/156 [==============================] - 355s 2s/step - loss: 0.0571 - acc: 0.9827 - val_loss: 1.1352 - val_acc: 0.6603
Epoch 7/20
156/156 [==============================] - 354s 2s/step - loss: 0.0488 - acc: 0.9853 - val_loss: 1.4201 - val_acc: 0.5871


In [16]:
resNet50_model.save('resNet50_model_fine_tuning_model.h5')

In [10]:
resNet50_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/resnet50_fine_tune'),
                                   CSVLogger('resnet50_fine_tune'),
                                   EarlyStopping(patience=5)])

Epoch 1/20
156/156 [==============================] - 354s 2s/step - loss: 0.0737 - acc: 0.9780 - val_loss: 1.8423 - val_acc: 0.5477
Epoch 2/20
156/156 [==============================] - 347s 2s/step - loss: 0.0668 - acc: 0.9802 - val_loss: 1.8539 - val_acc: 0.5375
Epoch 3/20
156/156 [==============================] - 354s 2s/step - loss: 0.0578 - acc: 0.9821 - val_loss: 1.8160 - val_acc: 0.5266
Epoch 4/20
156/156 [==============================] - 348s 2s/step - loss: 0.0555 - acc: 0.9835 - val_loss: 1.8495 - val_acc: 0.5301
Epoch 5/20
156/156 [==============================] - 351s 2s/step - loss: 0.0477 - acc: 0.9860 - val_loss: 1.7386 - val_acc: 0.5527
Epoch 6/20
156/156 [==============================] - 347s 2s/step - loss: 0.0448 - acc: 0.9870 - val_loss: 1.8002 - val_acc: 0.5301
Epoch 7/20
126/156 [=======================>......] - ETA: 1:03 - loss: 0.0428 - acc: 0.9880

KeyboardInterrupt: 